# Survey - Global Wheat Detection

- 小麦の穂の位置検出コンペ
- https://www.kaggle.com/c/global-wheat-detection

<img src="https://user-images.githubusercontent.com/846237/112433696-f4133b80-8d85-11eb-9f36-b6d20fdeab3e.png">

## 問題の概要

### データセット

- 世界各国の小麦が存在し，かなり見た目も異なる
- train:
    - 3,373枚(穂は147,761箇所)
- test:
    - 公開されているテストデータは10枚のみ
- paper
    - https://arxiv.org/pdf/2005.02162.pdf

### 評価指標

- IoU

## ❏ 予備知識

- 物体検出のおおまかな流れ
- Models
    - [Yolo v5](https://github.com/ultralytics/yolov5)
        - TODO(nishimori-m): スターターキットのリンクを追加する
    - EfficientDet
    - ResNeSt
        - 物体検出でのbackbone
            - cf. https://qiita.com/takoroy/items/07c5039ab12b74137626
- Toolkit
    - [Detectron2](https://github.com/facebookresearch/detectron2)
        - Facebook AI Researchによる物体検出ライブラリ
    - [MMDetection](https://github.com/open-mmlab/mmdetection)
        - 物体検出用のツールボックス
            - Faster RCNNやMask RCNNなど様々なベンチマーク用のアルゴリズムが使えるようになっている
- Post Processing
    - WBF(Weighted boxes fusion)
        - https://github.com/ZFTurbo/Weighted-Boxes-Fusion
        - 物体検出での矩形情報のアンサンブル方法
    - PL(Pseudo Labeling)
        - https://www.kaggle.com/ufownl/global-wheat-detection-pseudo-labaling
- EMA
- FPN
- Data Augmentations
    - clahe
        - ヒストグラムが均等になるようコントラストを調整
            - 暗いところの小麦の穂を見えやすくするため?
    - mixup
        - cf. [新たなdata augmentation手法mixupを試してみた](https://qiita.com/yu4u/items/70aa007346ec73b7ff05)
    - CoarseDropout
        - cf. https://www.kaggle.com/c/siim-isic-melanoma-classification/discussion/169721
     - GridMask
         - cf. https://ohke.hateblo.jp/entry/2020/06/27/230000
 
 <hr />

## よく利用されている物体検出アルゴリズム

Yolo v5を使えば間違いなさそう(2021年3月時点)

- Yolo v5
    - PyTorch
    - Keras
- EfficientDet
    - PyTorch
    - Keras
- Detectron2

<hr />

- 使われている画像サイズ
- data augmentation
- loss
- optimizer



## コンペ特有の知識

- ジグソーパズル
    - このコンペでは大きな画像を分割することでデータセットを作成している
        - つなぎ合わせることで，性能改善が可能
    - cf. https://github.com/lRomul/argus-tgs-salt/blob/master/mosaic/create_mosaic.py
    - 塩コンペでも同様のテクニックが有効だった
        - https://www.kaggle.com/c/tgs-salt-identification-challenge/overview
        
<hr />

## このコンペで効果的なこと

- 上位陣のモデルはYolo v5かEfficientDetのほぼ2択
- 検出時のしきい値を低く設定(0.1)するとスコアが改善
- 1stと2ndはData Augmentationと外部データの差が大きそう

## あまり役に立たなかったこと


## 注意事項

- MITライセンスのみ可ということで，性能が良いがGNUライセンスであるYolo v5を使って良いか結構揉めた

## ❏ EDA

## ❏ Top Solutions


- 1st - https://www.kaggle.com/c/global-wheat-detection/discussion/172418
    - code
        - https://github.com/rwightman/efficientdet-pytorch
    - dataset
        - image solution
            - 1024x1024 for EfficientDet D7(Fold 0,1,2,3,4)
            - 768x768 for EfficientDet D7(Fold 1, 3)
            - 512x512 for EfficientDet D5(Fold 4)
        - external data
            - https://www.kaggle.com/c/global-wheat-detection/discussion/164346)
            - https://plantimages.nottingham.ac.uk/
        - cleaning
            - 縦横のどちらかが10px以下の矩形は除外
    - models
        - <font color='red'>EfficientDet D5&D7</font>, <font color='red'>Faster RCNN FPN</font>
    - train
        - batch size
            - 8: EfficientDet D7(image size: 768)
            - 20: Faster FPN ResNet 152(image size: 1024)
        - optimizer
            - adam for EfficientDet
                - learning rate: 5e-4
            - sgd for FasterRCNN
                - learning rate: 5e-3
        - LR scheduler:
            - cosine-annealing
        - augmentation
            - Custom mosaic augmentation
            - MixUp
    - ensemble
        - 8 TTA(vflip x hflip x rotate90)

- 2nd - https://www.kaggle.com/c/global-wheat-detection/discussion/175961
    - code
        - https://github.com/liaopeiyuan/TransferDet
    - model
        - <font color='red'>EfficientDet - D6</font>(pretrained w/ COCO dataset)
            - D4〜D7まで試してD6が性能が一番良かった
    - dataset
        - image resolution: 1024x1024
    - train
        - augmentation
            - crop, hue, random brightness/contrast
            - to gray
            - vertical and horizontal flip
            - random rotate 90
            - cutout

- 3rd - https://www.kaggle.com/c/global-wheat-detection/discussion/179055
    - code
        - https://github.com/ufownl/global-wheat-detection
    - model
        - <font color='red'>Yolo v5</font>
            - backbone: Darknet 53
    - train
        - max epochs: 9
        - learning rate: 0.001
        - batch size: 8
        - image resolution: 512x512
        - threshold: 0.1(物体検出のためのしきい値)
        - pseudo labeling
            - cf. https://www.kaggle.com/ufownl/global-wheat-detection-pseudo-labaling
    - post processing
        - TTAにWBF(Weighted boxes fusion)を利用

- 5th - https://www.kaggle.com/c/global-wheat-detection/discussion/172458
    - blog(日本語)
        - https://acro-engineer.hatenablog.com/entry/2020/08/21/175617
    - dataset
        - image resolution: 1024x1024
    - models
        - <font color='red'>EfficientDet D3 - D5</font>
    - train
        - 5 folds
        - optimizer: AdamW
        - max epoichs: 100
        - batch size: 640
        - pseudo labeling
        - augmentation:
            - mixup, mosaic, scale
            - hue, brightness
            - grid mask
                - cf. https://arxiv.org/abs/2001.04086
    - misc
        - threshold: 検出時のしきい値を0.1にすれば，0.695出ていたらしい

- 9th - https://www.kaggle.com/c/global-wheat-detection/discussion/172569
    - code: 
        - https://github.com/amirassov/kaggle-global-wheat-detection
    - dataset
        - external data
            - https://www.kaggle.com/c/global-wheat-detection/discussion/164346
   　- model
        - <font color='red'>MMDetection</font> 
            - cf. https://github.com/open-mmlab/mmdetection
    - train:
        - augmentaion
            - hflip, scale, shift, rotate90
            - brightness/contrast, hue saturation, rgb shift
            - random gamma
            - clahe
            - blur, motion blur
            - gauss noise
            - image compression
            - coarse dropout

- 16th - https://www.kaggle.com/c/global-wheat-detection/discussion/172567
    - train
        - augmentaiton
            - scale, mosaic, mixup, crop, cutout, hue, brightness, clahe

## ❏ Top Notebooks

- 2nd, https://www.kaggle.com/alexanderliao/effdet-d6-pl-s-bn-r-bb-a3-usa-eval-94-13-db
    - EfficientDet - D6
- 3rd, https://www.kaggle.com/x2x21x21x21/3rd-place-solution
- 6th - https://www.kaggle.com/stonewst98/what-a-pity-only-0-0001-away-from-0-77
    - Effficient Det
- 9th - https://www.kaggle.com/amiras/pseudo-ensemble-detectors-3-st-universenet-r10
- 13th - https://www.kaggle.com/dpyrtfq2372/efficientdet-with-double-psudo-labeling

## ❏ Key ideas

### Custom mosaic augmentation

- cf. https://www.kaggle.com/c/global-wheat-detection/discussion/172418
- 1stの方が試した方法
- CutMixの代わりに利用


### Mixup


- cf. https://www.kaggle.com/c/global-wheat-detection/discussion/172418

### Pseudo labeling



<hr>

## まとめ - 物体検出で性能を上げる方法

- Yolo v5を使う
- Pseudo Labelingを使う

## 疑問

- 胸部X線でもMixupって使える？(画像を縦半分とかでぶった切ってつないでもOK?)

## References

- https://www.kaggle.com/nvnnghia/yolov5-pseudo-labeling/data
    - Yolo v5 + Pseudo Labelingで高スコアを記録しているコード

## TODO

- 各種の項目を別々に書くだけだと本家を見たほうが早いしわかりやすいだけなので意味がない．
     - 機能ごとに区切ってどのように使われているかを調べまとめるようにする
     - 自分がコードをスクラッチで組む際に使いやすいフォーマットにする